<a href="https://colab.research.google.com/github/RachelJKim/TaCo/blob/main/TaCo_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install svgwrite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 891.1 kB/s eta 0:00:00


In [19]:
import cv2
import numpy as np
import math
import svgwrite
from sklearn.cluster import MiniBatchKMeans
from scipy.spatial import cKDTree


class FlatImageSegmenter:
    def __init__(self, image, k=8, hsv_threshold=0.03, area_threshold=100, kernel_size=(5, 5)):
        self.img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        self.k = k
        self.hsv_threshold = hsv_threshold
        self.area_threshold = area_threshold
        self.kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
        self.dominant_colors = None
        self.img_simplified = None
        self.segments = []

    def _cluster_colors(self):
        pixels = self.img.reshape(-1, 3).astype(np.float32)
        mb_kmeans = MiniBatchKMeans(n_clusters=self.k, random_state=42).fit(pixels)
        centers = mb_kmeans.cluster_centers_
        labels = mb_kmeans.labels_
        pixel_counts = np.bincount(labels, minlength=self.k)
        return centers, pixel_counts

    def _merge_clusters(self, centers, pixel_counts):
        tree = cKDTree(centers)
        merged_centers = []
        merged_indices = set()

        for i, center in enumerate(centers):
            if i in merged_indices:
                continue

            # Find all clusters within the threshold distance
            indices = tree.query_ball_point(center, r=self.hsv_threshold)
            indices = [idx for idx in indices if idx not in merged_indices]

            # Merge clusters
            if indices:
                merged_indices.update(indices)
                weights = pixel_counts[indices]
                weighted_centers = centers[indices] * weights[:, None]
                merged_center = np.sum(weighted_centers, axis=0) / np.sum(weights)
                merged_centers.append(merged_center)
            else:
                merged_centers.append(center)

        return np.array(merged_centers)

    def get_dominant_colors(self):
        colors, pixel_counts = self._cluster_colors()
        self.dominant_colors = self._merge_clusters(colors, pixel_counts)
        return self.dominant_colors

    def create_simplified_image(self):
        self.get_dominant_colors()
        h, w, d = self.img.shape
        pixels = self.img.reshape(-1, 3)
        tree = cKDTree(self.dominant_colors)
        _, indices = tree.query(pixels)
        new_colors = self.dominant_colors[indices].astype(np.uint8)
        self.img_simplified = new_colors.reshape(h, w, d)
        return self.img_simplified, indices

    def segment_image(self):
        _, indices = self.create_simplified_image()

        for i, center in enumerate(self.dominant_colors):

          # Avoid negative errors (TODO: Find why this happens)
          center[0] = np.clip(center[0], 0, 179).astype(np.uint8)
          center[1] = np.clip(center[1], 0, 255).astype(np.uint8)
          center[2] = np.clip(center[2], 0, 255).astype(np.uint8)

          mask = np.zeros(self.img_simplified.shape[:2], dtype=np.uint8)
          mask[indices.reshape(self.img_simplified.shape[:2]) == i] = 255


          mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, self.kernel)
          mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, self.kernel)


          contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

          # Filter out small contours
          contours = [cnt for cnt in contours if cv2.contourArea(cnt) > self.area_threshold]

          self.segments.append((center, mask, contours))

        return self.segments



def find_color_regions(img, kernel, area_threshold):
    """Find distinct color regions in the image with improved logic for simplified images."""
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Filter contours by area threshold
    filtered_contours = [c for c in contours if cv2.contourArea(c) > area_threshold]
    return filtered_contours


def generate_svg(segments):
    """Generate SVG content with customized pattern fills based on hue."""
    dwg = svgwrite.Drawing()

    for index, (color, _, contours) in enumerate(segments):
        h, s, v = color

        # Skip the white regions
        if s < 30 and v > 220:
            continue

        pattern_size = 40
        pattern_id = f"clockPattern{index}"

        pattern = create_pattern(dwg, h, pattern_size, pattern_id)
        dwg.defs.add(pattern)

        fill_color = f"url(#{pattern_id})" if v > 50 else "black"

        for contour in contours:
            path_data = "M " + " L ".join(f"{point[0][0]} {point[0][1]}" for point in contour) + " Z"
            dwg.add(dwg.path(d=path_data, fill=fill_color, stroke="black", stroke_width="7"))

    return dwg.tostring()



def create_pattern(dwg, hue, pattern_size, pattern_id):
    """Generate a clock-shaped pattern based on the hue."""

    pattern = dwg.pattern(id=pattern_id, size=(pattern_size, pattern_size), patternUnits="userSpaceOnUse")
    pattern.add(dwg.rect(insert=(0, 0), size=(pattern_size, pattern_size), fill="white"))

    hand_length = pattern_size / 2 * 0.8

    # Fixed hand - horizontal
    pattern.add(dwg.line(start=(pattern_size / 2 - hand_length, pattern_size / 2),
                         end=(pattern_size / 2 + hand_length, pattern_size / 2),
                         stroke="black", stroke_width=3))

    # Rotating hand
    angle = (hue / 360) * 360  # Hue to Angle

    end_x = pattern_size / 2 + hand_length * math.cos(math.radians(angle - 90))
    end_y = pattern_size / 2 + hand_length * math.sin(math.radians(angle - 90))
    pattern.add(dwg.line(start=(pattern_size / 2, pattern_size / 2),
                         end=(end_x, end_y),
                         stroke="black", stroke_width=3))

    return pattern



def raster_to_svg(img, svg_path, k=8, hsv_threshold=0.03, area_threshold=100, kernel_size=(5, 5)):
    """Convert a raster image to an SVG with distinct color regions using FlatImageSegmenter."""
    segmenter = FlatImageSegmenter(img, k, hsv_threshold, area_threshold, kernel_size)
    segmenter.create_simplified_image()  # Quantize and simplify the image
    segments = segmenter.segment_image()  # Segment the simplified image

    svg_content = generate_svg(segments)
    with open(svg_path, 'w') as f:
        f.write(svg_content)


In [3]:
def convert_transparent_to_white(img):
    if img.shape[2] == 4:
        print("yes")
        background = np.ones_like(img[:, :, :3], dtype=np.uint8) * 255
        foreground = img[:, :, :3]
        alpha = img[:, :, 3]
        foreground = cv2.bitwise_and(foreground, foreground, mask=alpha)
        background = cv2.bitwise_and(background, background, mask=cv2.bitwise_not(alpha))
        img = cv2.add(foreground, background)
    return img

def resize_image(img, target_size=1500):
    larger_dimension = max(img.shape[:2])

    ratio = target_size / larger_dimension

    new_width = int(img.shape[1] * ratio)
    new_height = int(img.shape[0] * ratio)

    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    return resized_img

# Test the Code

In [4]:
from google.colab import files
import cv2
import numpy as np
from io import BytesIO

# Upload file
uploaded = files.upload()
filename = next(iter(uploaded))

# Read the image file
image_data = uploaded[filename]
image_array = np.frombuffer(image_data, np.uint8)
img = cv2.imdecode(image_array, cv2.IMREAD_UNCHANGED)
img = convert_transparent_to_white(img)
img_resized = resize_image(img, target_size=1500)

Saving google1.png to google1.png
yes


In [20]:
raster_to_svg(img_resized, "output.svg")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
